In [3]:
from collections import deque

MOVES = {
    "Up": (-1, 0),
    "Down": (1, 0),
    "Left": (0, -1),
    "Right": (0, 1)
}

def get_blank_position(state):
    """Find the row and column of the blank (0) tile."""
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def generate_neighbors(state):
    """Generate all possible moves from the current state."""
    neighbors = []
    x, y = get_blank_position(state)

    for move, (dx, dy) in MOVES.items():
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 3 and 0 <= new_y < 3:  # Ensure move is within bounds
            new_state = [row[:] for row in state]  # Copy state
            # Swaping blank with adjacent tile
            new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
            neighbors.append((new_state, move))

    return neighbors

def bfs(initial_state, goal_state):
    visited = set()  # To keep track of visited nodes
    queue = deque()

    visited.add(start)
    queue.append(start)

    while queue:
        node = queue.popleft()

        if state == goal_state:
            return path

        state_tuple = tuple(tuple(row) for row in state)  # to convert to tuple
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

# Example usage
initial_state = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

solution = bfs(initial_state, goal_state)

if solution:
    print(f"Solution found in {len(solution)} moves: {solution}")
else:
    print("No solution exists.")


NameError: name 'start' is not defined